# Дизайн кармана связывания

*весь код - для запуска в колабе. перед запуском включите gpu*

## 1. Постановка задачи

Будем генерировать структуру, способную связывать дигоксигенин. В качестве стартовой структуры возьмем комплекс искусственно созданного белка с дигоксигенином (https://www.rcsb.org/structure/1LKE)

Загрузим файл со структурой и визуализируем ее.

In [114]:
! wget https://files.rcsb.org/download/1LKE.pdb1.gz
! gzip -d 1LKE.pdb1.gz

--2023-12-13 16:04:45--  https://files.rcsb.org/download/1LKE.pdb1.gz
Resolving files.rcsb.org (files.rcsb.org)... 128.6.159.157
Connecting to files.rcsb.org (files.rcsb.org)|128.6.159.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29060 (28K) [application/octet-stream]
Saving to: ‘1LKE.pdb1.gz’

1LKE.pdb1.gz        100%[===================>]  28.38K  --.-KB/s    in 0.03s   

2023-12-13 16:04:45 (1.02 MB/s) - ‘1LKE.pdb1.gz’ saved [29060/29060]

gzip: 1LKE.pdb1 already exists; do you wish to overwrite (y or n)? ^C


In [115]:
!pip install py3Dmol
!pip install Bio

from Bio import PDB
from Bio.PDB.PDBParser import PDBParser
import py3Dmol

from IPython.display import clear_output

clear_output()

In [ ]:
parser=PDBParser()
structure=parser.get_structure("1LKE",'1LKE.pdb1')
for model in structure:
    for chain in model:
        for residue in chain:
            if residue.get_resname()!='HOH':
                print(chain.id, residue.id[1], residue.get_resname())

In [117]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/1LKE.pdb1','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'chain':'A','resn':'DOG'},{'stick':{'colorscheme':'grayCarbon'}})

p.zoomTo() # центрировать структуру
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 2. Диффузии

Используем диффузионную модель для генерации белка RFdiffusion (https://github.com/RosettaCommons/RFdiffusion)

In [ ]:
%%time
import os, time, signal
import sys, random, string, re
if not os.path.isdir("params"):
  os.system("apt-get install aria2")
  os.system("mkdir params")
  # send param download into background
  os.system("(\
  aria2c -q -x 16 https://files.ipd.uw.edu/krypton/schedules.zip; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/5532d2e1f3a4738decd58b19d633b3c3/ActiveSite_ckpt.pt; \
  aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar; \
  tar -xf alphafold_params_2022-12-06.tar -C params; \
  touch params/done.txt) &")

if not os.path.isdir("RFdiffusion"):
  print("installing RFdiffusion...")
  os.system("git clone https://github.com/sokrypton/RFdiffusion.git")
  os.system("pip -q install jedi omegaconf hydra-core icecream pyrsistent")
  os.system("pip install dgl==1.0.2+cu116 -f https://data.dgl.ai/wheels/cu116/repo.html")
  os.system("cd RFdiffusion/env/SE3Transformer; pip -q install --no-cache-dir -r requirements.txt; pip -q install .")
  os.system("wget -qnc https://files.ipd.uw.edu/krypton/ananas")
  os.system("chmod +x ananas")

if not os.path.isdir("RFdiffusion/models"):
  print("downloading RFdiffusion params...")
  os.system("mkdir RFdiffusion/models")
  models = ["Base_ckpt.pt","Complex_base_ckpt.pt","ActiveSite_ckpt"]
  for m in models:
    while os.path.isfile(f"{m}.aria2"):
      time.sleep(5)
  os.system(f"mv {' '.join(models)} RFdiffusion/models")
  os.system('wget  -P RFdiffusion/models -q')
  os.system("unzip schedules.zip; rm schedules.zip")

if 'RFdiffusion' not in sys.path:
  os.environ["DGLBACKEND"] = "pytorch"
  sys.path.append('RFdiffusion')

Запустим RFDiffusion.

In [ ]:
%%time
! RFdiffusion/run_inference.py \
inference.input_pdb=1LKE.pdb1 \
'contigmap.contigs=[166-166]' \
inference.output_prefix=result/simple_design inference.num_designs=1

Визуализируем результат

In [120]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/result/simple_design_0.pdb','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'chain':'A','resn':'DOG'},{'stick':{'colorscheme':'grayCarbon'}})

p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Получился бессмысленный результат. Почему?

## 3. Скаффолдинг мотива связывания

По умолчанию диффузионная модель не видит лиганды, отличные от полипептидов. Чтобы это изменить, надо задать дополнительные потенциалы. Для этого определим мотив, участвующий в связывании лиганда.

In [121]:

lig_atoms=[] # добавим атомы лиганда в список
for model in structure:
    for chain in model:
        for residue in chain:
            if residue.get_resname()=='DOG':
                lig_atoms.extend(residue.get_atoms())
                break

lresi=[] # найдем остатки, атомы которых ближе чем 3 ангстрема к атомам лиганда
for model in structure:
    for chain in model:
        for residue in chain:
            if residue.get_resname()=='HOH':
                continue
            elif residue.get_resname()=='DOG':
                continue
            for atom in residue:
                if any([(atom-x)<3 for x in lig_atoms]):
                    lresi.append(residue.get_id()[1])
                    break
lresi

[35, 47, 86]

Определим мотив связывания, а остальные участки белка сгенерируем с нуля.

In [122]:
lresi=[33,34,35,36,37,45,46,47,48,49, 84,85, 86, 87,88] # добавим к взаимодействующим остаткам по 2 остатка с обеих сторон

In [ ]:
#Задание: написать скрипт, котопый будет генерировать строку для contigs



In [ ]:
%%time
! RFdiffusion/run_inference.py \
inference.input_pdb=1LKE.pdb1 \
'contigmap.contigs=[32-32/A33-37/7-7/A45-49/34-34/A84-88/77-77]' \
inference.output_prefix=result/aux_pot inference.num_designs=1 \
potentials.guide_scale=1  \
'potentials.guiding_potentials=["type:substrate_contacts,s:1,r_0:8,rep_r_0:5.0,rep_s:2,rep_r_min:1"]' \
potentials.substrate=DOG \
inference.ckpt_override_path=RFdiffusion/models/ActiveSite_ckpt.pt

In [127]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/1LKE.pdb1','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'resi':[33,34,35,36,37,45,46,47,48,49,84,85,86,87,88]},{'cartoon':{'color':'yellow'}})
p.setStyle({'chain':'A','resn':'DOG'},{'stick':{'colorscheme':'grayCarbon'}})

p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [125]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/result/aux_pot_0.pdb','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'resi':[33,34,35,36,37,45,46,47,48,49,84,85,86,87,88]},{'cartoon':{'color':'yellow'}})
p.setStyle({'chain':'A','resn':'DOG'},{'stick':{'colorscheme':'grayCarbon'}})

p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Как можно вставить лиганд в финальную структуру? Наложить друг на друга атомы мотива в оригинальной и задизайненной структуре.

In [128]:
from Bio.PDB.Superimposer import Superimposer
from Bio.PDB.PDBIO import PDBIO

Читаем из оригинальной структуры атомы мотива и лиганда.

In [129]:
orig_atoms=[]
for model in structure:
    for chain in model:
        for residue in chain:
            if residue.get_id()[1] in lresi:
                for atom in residue:
                    if atom.get_name() in ['CA','C','N','O']:
                        orig_atoms.append(atom)
            elif residue.get_resname()=='DOG':
                ligand=residue



Читаем соответстующие атомы мотива из дизайнерской структуры

In [130]:
redis=parser.get_structure("redis",'result/aux_pot_0.pdb')
redis_atoms=[]
for model in redis:
    for chain in model:
        for residue in chain:
            if residue.get_id()[1] in lresi:
                for atom in residue:
                    redis_atoms.append(atom)

/usr/local/lib/python3.10/dist-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/usr/local/lib/python3.10/dist-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/usr/local/lib/python3.10/dist-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/usr/local/lib/python3.10/dist-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'O' for Atom (name=O) with given element ''
  warnings.warn(msg, PDBConstructionWarning)


Делаем наложение, т.е. определяем ротации и трансляции, необходимые для оптимального совмещения атомов, и применяем их к атому лиганда.

In [131]:
sup=Superimposer()
sup.set_atoms(redis_atoms, orig_atoms)
sup.apply(ligand)

Добавляем повернутый нужным образом лиганд в дизайнерскую структуру и сохраняем.

In [132]:
redis[0]['A'].add(ligand)

io = PDBIO()
io.set_structure(redis)
io.save("result/aux_pot_0_add_lig.pdb")

In [144]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/result/aux_pot_0_add_lig.pdb','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'resi':[33,34,35,36,37,45,46,47,48,49,84,85,86,87,88]},{'cartoon':{'color':'yellow'}})
p.setStyle({'chain':'A','resn':'DOG'},{'stick':{'colorscheme':'grayCarbon'}})

p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 3. Motif scaffolding с помошью RFDesign

Используем RFDesign для решения той же задачи

In [ ]:
# get repo and install dependencies
!git clone https://github.com/RosettaCommons/RFDesign.git
!pip -q install torch-geometric lie_learn icecream e3nn py3Dmol omegaconf
!pip install dgl==1.0.2+cu116 -f https://data.dgl.ai/wheels/cu116/repo.html

# get weights
!wget http://files.ipd.uw.edu/pub/rfdesign/weights/BFF_last.pt -P RFDesign/hallucination/weights/rf_Nov05/
!wget http://files.ipd.uw.edu/pub/rfdesign/weights/BFF_mix_epoch25.pt -P RFDesign/inpainting/weights/

# get pyrosetta for sidechain relaxation
!gdown 1Pts0USd16GAtqz5v_4ErTu_7YBpysF9G
!pip install pyrosetta-2023.45+release.a6d9ba8-cp310-cp310-linux_x86_64.whl

clear_output()

Для использования дополнительных лоссов лиганд должен быть в отдельном файле.

In [136]:
io = PDBIO()
io.set_structure(ligand)
io.save("ligand.pdb")

Запустим дизайн

In [ ]:
%%time
!mkdir rfdesign
!python RFDesign/hallucination/hallucinate.py \
--pdb 1LKE.pdb1 --contigs=A33-37,A45-49,A84-88 --len=166-166 \
 --steps=g100,m100 \
--w_rep 1 --w_atr 1 --rep_pdb ligand.pdb --atr_pdb ligand.pdb \
--num 1 --out rfdesign/result


Добавим боковые цепи

In [ ]:
%%time
!python RFDesign/scripts/RosettaTR/trfold_relax.py --roll -sg 7,3 rfdesign/result_0.npz rfdesign/result_0.pdb rfdesign/result_0_relaxed.pdb

In [ ]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/rfdesign/result_0_relaxed.pdb','r').read(),'pdb')
p.setStyle({'chain':'A'},{'cartoon':{'color':'green'}})
p.setStyle({'chain':'A','resn':'DOG'},{'stick':{'colorscheme':'grayCarbon'}})

p.zoomTo()
p.show()

## 4. Домашняя работа

1. Восстановить последовательность сгенерированной структуры одним из способов, рассмотренных на предыдущем занятии (например, MPNNN FastRelax)
